**🔍 Splunk Windows Security Log Analysis Project**

🎯 Project Goal
To investigate Windows Security Event Logs using Splunk in order to detect indicators of compromise (IOCs) and develop a simple threat detection framework that could be used in a real SOC environment. The project will simulate how a security analyst would triage endpoint activity using Windows logs.

This is a three-phase project:

Establish the normal baseline of Windows activity

Identify suspicious or abnormal behaviors

Build basic detection logic or alerting rules


**Phase 1 – Baseline System Activity**

**Objective:**  
Establish what "normal" activity looks like on the system by analyzing logon behavior captured in native Windows Security logs. This baseline is critical because it defines the standard operating environment: who typically accesses the system, what types of logons occur, when those logons happen, and how the system behaves under normal conditions. Without this understanding, it’s impossible to reliably detect anomalies or intrusions in later phases.

In this phase, we perform three focused analyses. First, we identify which accounts are logging in, what type of logon they’re using (such as interactive, remote, or service-based), and from which host. This tells us who is active on the machine and whether that access is consistent with expectations. Next, we analyze how logon types vary over time, allowing us to recognize patterns and detect unusual spikes in behavior that might suggest suspicious activity. Finally, we take a broader look across all Windows Security event codes to understand what kinds of activity the system is generating most frequently. This event inventory gives us a foundation for recognizing rare, unexpected, or high-risk behaviors as we move into threat detection in later phases.

Before running any specific queries, it's important to understand what kind of language Splunk uses to retrieve and analyze data. Splunk relies on its own query language called **SPL**, which stands for **Search Processing Language**. SPL is designed to filter, transform, and visualize event data stored in Splunk indexes. It allows analysts to search through massive amounts of machine data quickly and flexibly. The basic structure of an SPL query includes: (1) a search filter, such as a specific index, sourcetype, or event code; (2) a pipeline of commands using pipes (`|`) to transform or group the results; and (3) optional commands to format or visualize the data (e.g., `table`, `stats`, `timechart`). While SPL can become complex, most practical queries are composed of readable, modular steps. In this project, AI-generated SPL will be used to drive analysis, but understanding its structure gives better insight into how filters and logic extract exactly the data we need.

The first SPL query we use in this project is designed to show **who is logging in**, **what type of logon they're using**, and **from which machine**. Here's how it works, broken down by each component:

index=main sourcetype=WinEventLog:Security EventCode=4624
| stats count by Account_Name, Logon_Type, Workstation_Name

index=main
This tells Splunk to search the "main" index, which is the location where incoming log data is stored. An index in Splunk is like a database or storage bucket for a category of logs. You can have multiple indexes (e.g., "main", "firewall", "windows", etc.), but in this project, everything is in "main".

sourcetype=WinEventLog:Security
This narrows the search to only events from the Windows Security Log. The sourcetype is how Splunk labels the kind of data a log comes from — in this case, it’s structured Windows event data, not generic logs.

EventCode=4624
This filters for successful logon events. Every Windows Security Event has a numeric code that represents a specific action. EventCode 4624 is generated whenever someone (or something) successfully logs into the system.

| stats count by Account_Name, Logon_Type, Workstation_Name
This part of the query uses a "stats" command to group the results. It counts how many times each combination of:

Account_Name (the username or system account)

Logon_Type (how they logged in — e.g., 2 = interactive, 5 = service)

Workstation_Name (which machine the logon came from)
appeared in the logs.

Together, this SPL query builds a simple but powerful picture of normal access patterns: who logs in, how, and where from. It lays the groundwork for detecting abnormal users, unexpected logon types (like remote or batch), or logons from machines that aren’t part of normal workflows.

**🔍 Query 1: Who logs in, how, and where from?**

**SPL:**
index=main sourcetype=WinEventLog:Security EventCode=4624
| stats count by Account_Name, Logon_Type, Workstation_Name

**Key Results**

Account_Name	Logon_Type	Workstation_Name	Count
Steve	           2	     DESKTOP-UPKRV33	2
SYSTEM	           5	      -	                80
DESKTOP-UPKRV33$	2, 5 - / DESKTOP-UPKRV33	85
DWM-2 / UMFD-2	    2	      -	                 3

Interpretation:

Steve logging in with Logon Type 2 confirms interactive user activity is captured.

SYSTEM and $ accounts with Logon Type 5 reflect service and machine-level logons.

No unexpected accounts observed.

**Understanding What This Data Represents**

The data shown in Query 1 may look overly complex at first, but it reflects common and expected behavior on virtually all Windows systems. Even when a human user is not actively logging in, the operating system itself is constantly performing background logon operations to start services, maintain internal processes, and authenticate accounts.

The most frequent entries in our results include:
- `DESKTOP-UPKRV33$` with Logon Types 2 and 5
- `SYSTEM` with Logon Type 5

Let’s break this down.

**What does the dollar sign ($) mean?**  
The dollar sign at the end of `DESKTOP-UPKRV33$` indicates that this is a **machine account**, not a human user. In Windows, every computer on a network has its own account in the system's security structure. These accounts are used internally when the system authenticates itself to services — for example, when the machine checks in with the domain, starts Windows Update, or initiates a network connection.

**Why are there logons from this account with Logon Type 2 or 5?**  
- `Logon Type 5` indicates a **service logon**. This happens constantly as Windows services start and stop (e.g., security, networking, system maintenance).
- `Logon Type 2` indicates an **interactive logon** — typically a human logging in at the keyboard. But when you log in as a user, your machine may also establish an interactive logon under its own system context (`DESKTOP-UPKRV33$`) to perform setup tasks in parallel.

**What is the `SYSTEM` account?**  
This is the core **Windows internal operating account**. It runs system-level processes like the kernel, service controllers, and hardware interface tasks. When we see `SYSTEM` logging in with Logon Type 5, it means Windows is activating services or performing internal functions.

**Why does this matter to a SIEM like Splunk?**  
These logon events represent the **normal baseline** of a Windows machine's background activity. A SIEM like Splunk doesn’t just record what humans do — it records everything the system does. So even when the system is idle, these events are generated continuously.

This is why our first query is so important: it gives us a clear picture of what "normal noise" looks like. Without understanding this, we couldn’t possibly know when something abnormal (like a remote logon or unexpected user activity) occurs.

In summary, these log entries aren't suspicious — they're expected. They reflect the machine authenticating itself, starting services, and operating normally. Every Windows system would show similar patterns unless it was heavily customized or restricted.

**Query 2: When do logons occur, and what types?**

**SPL:**
index=main sourcetype=WinEventLog:Security
| stats count by EventCode

➡ Inventory of common event types (baseline signal types).

Key Results (summary):

Hour (Start of Interval)	Logon_Type=2 (Interactive)	Logon_Type=5 (Service)
2025-06-16 08:00	         5	                         32
2025-06-15 09:00	         0	                         20
2025-06-15 10:00	         0	                         10
2025-06-15 11:00	         0	                         17
All other hours	             0	                         0–1

Interpretation:

Logon activity is primarily driven by background services (Logon_Type=5)

Human interactive logons (Logon_Type=2) occurred only during a brief window (5 events around 8:00 AM on June 16), matching expected user login behavior

No logons occurred during off-hours, which reflects a quiet, low-risk system usage pattern — useful for establishing a clean baseline

**Contextual Perspective: Query 2 in Relation to Query 1**

The results of Query 2 reinforce and extend what we established in Query 1. In the earlier query, we identified the key accounts involved in logon activity, including the human user (`Steve`), system-level accounts (`SYSTEM`), and the machine account (`DESKTOP-UPKRV33$`). Query 2 adds a temporal layer to that picture, showing that these logons occur in predictable hourly patterns, with service logons occurring regularly and interactive logons confined to specific user activity periods. This supports our baseline by confirming that not only are the right accounts logging in, but they are doing so at expected times and in expected volumes. Together, the two queries provide both a static and dynamic view of normal system behavior.


**Query 3: What types of Security events are most common?**

**SPL:**

index=main sourcetype=WinEventLog:Security
| stats count by EventCode
| sort -count

Top Results:

EventCode	              Count	                              Description (summary)
4688	                  30,444	                          Process creation (very detailed + noisy)
4689	                  6,215	                              Process end (paired with 4688)
5379	                  2,038	                              Credential Manager read attempt
5447	                  828	                              Windows Filtering Platform (network connection)
4703	                  656	                              Rights assignment changed
4670	                  299	                              Object permissions change
4624	                  96	                              Successful logon
4672	                  94	                              Special privileges assigned at logon (e.g. admin)
5449	                  84	                              WFP callout change (low-level network logging)

Interpretation:

The system is logging a large volume of process creation events (4688), which is excellent for detailed behavioral analysis and future threat hunting

Logons (4624) and special privilege logons (4672) are being logged properly, confirming Phase 1 baseline is working

EventCodes like 5379 and 5447 indicate credential access and network flow visibility, which will be useful in Phase 2

Low-frequency events (e.g., 4776, 4695) may be relevant for rare or targeted activity and should be watched during IOC triage

**Contextual Perspective: Query 3 in Relation to Queries 1 and 2**

While Queries 1 and 2 focused specifically on logon behavior, Query 3 broadens our perspective by showing what kinds of security events the system is generating most frequently overall. This is important because it helps us understand the volume and types of data the system naturally produces, placing our earlier findings into a wider context. For example, we can now see that process creation events (EventCode 4688) dominate the dataset, which is normal for an active Windows machine and reinforces that much of the background noise in Query 2’s time-based logon activity likely comes from service-level operations. Additionally, seeing EventCode 4624 appear here with a modest count gives us confidence that our logon analysis is well-targeted and representative. Query 3 doesn’t just inventory events — it confirms that what we’ve been looking at so far (logons) is meaningful and proportionally accurate compared to the full scope of system activity. This completes our Phase 1 baseline with a solid understanding of both targeted logon behavior and the broader event landscape.

**Operational Perspective: How Baseline Analysis is Used in Real SOC Environments**

Establishing a baseline of system activity — like we did in Phase 1 — is a fundamental part of security operations in any modern SOC, but it's not typically done manually on a daily basis. Instead, baselining is often performed during initial system deployment, during major configuration changes, or as part of a tuning process when new monitoring rules are created. Most enterprise-grade SIEMs use a combination of historical data, machine learning models, and threshold-based logic to build and maintain dynamic baselines automatically. These baselines track what "normal" looks like for users, hosts, logon types, and time-of-day patterns. If activity significantly deviates from those learned norms (for example, an account logging in remotely at 3 AM for the first time), it can generate an alert. So while a human analyst doesn’t manually re-establish a baseline every day, understanding what the baseline should look like is essential — both for building detection logic and for investigating whether an alert is truly anomalous or just part of regular operations. The kind of hands-on baseline analysis we performed here is exactly what junior SOC analysts are expected to understand and interpret when triaging alerts or reviewing user behavior.




**🚩 PHASE 2 – Suspicious Behavior Investigation**
📌 Objective
Query the data for known suspicious events such as:

Brute force attempts

Unauthorized account creation

Group membership escalation

**SPL Queries**

A. Failed Logon Attempts (Possible Brute Force)

index=main sourcetype=WinEventLog:Security EventCode=4625
| stats count by Account_Name, Source_Network_Address
| sort -count

Key Results:

Account_Name	     Source_Network_Address	      Count
DESKTOP-UPKRV33$	   127.0.0.1	               1
Steve	               127.0.0.1	               1

Interpretation:

Two failed logon attempts occurred within the past 7 days.

Both originated locally from the system itself (127.0.0.1), not from a remote host.

The failed logon by Steve may have been a mistyped password or other benign event.

The failed logon by DESKTOP-UPKRV33$ likely reflects a background service or process that attempted to authenticate unsuccessfully.

No signs of brute-force attempts, repeated failures, or suspicious remote IPs are present — a normal and low-risk pattern.





**B. New Account Creation or Group Modifications**

index=main sourcetype=WinEventLog:Security EventCode=4720 OR EventCode=4728
| table _time, Target_Account_Name, Account_Name, EventCode

Key Results:
No results found, even when searching across the full available log history.

Interpretation:

The system shows no signs of local account creation or privilege escalation activity.

This is expected in a well-maintained single-user or lab environment.

In an enterprise SOC setting, a sudden appearance of EventCode 4720 or 4728 — especially outside business hours or by unexpected accounts — would be a high-priority alert.




Query C: Were there any logons outside of business hours?

**SPL:**
in sourcetype=WinEventLog:Security EventCode=4624
| eval hour=strftime(_time,"%H")
| where hour<7 OR hour>19
| stats count by Account_Name, hour


Key Results (selected):

Account_Name	    Hour	                 Count
DESKTOP-UPKRV33$	00–06, 20–23	     High counts (10–100+) across off-hours
SYSTEM	            00–06, 20–23	     Mirrors machine account — expected behavior
Steve	            02, 05, 06, 20	     Small number of interactive logons (2–4 per hour)
DWM-1, UMFD-*	    20	                 1–2 each — normal Windows system accounts

Interpretation:

The majority of off-hours logon activity is from system accounts (SYSTEM, DESKTOP-UPKRV33$), which is expected and reflects scheduled tasks or background services.

Steve (the human account) shows very limited off-hours activity — only small bursts at 2 AM, 5 AM, 6 AM, and 8 PM. These could be tied to system start-up, manual testing, or normal usage patterns.

No evidence of high-volume or unusual off-hours logon behavior from unexpected accounts.

In a SOC environment, sudden off-hours logons by privileged or rarely-used accounts would trigger alerts. Here, the pattern appears normal and consistent with your own usage and system activity.


**Phase 2 Summary: Suspicious Behavior Investigation**

In Phase 2, we shifted from baseline activity to targeted investigation, using SPL queries designed to uncover signs of abnormal or unauthorized behavior. Query A filtered for failed logon attempts (EventCode 4625), which could indicate brute-force attacks or password guessing. We observed only two failures — one from the user account `Steve` and one from the machine account — both originating from the local system (`127.0.0.1`) and showing no signs of attack. Query B examined account creation and privilege escalation events (EventCodes 4720 and 4728), and returned no results, confirming that no new accounts were added or granted elevated access. Query C analyzed successful logons (EventCode 4624) that occurred outside of business hours, revealing predictable and low-volume activity from system-level accounts and minimal, explainable activity from the user account. Collectively, these findings reinforce that the system is operating in a stable, low-risk state with no evidence of interactive misuse, escalation attempts, or unauthorized access patterns.



**PHASE 3 – Detection Logic**
Objective
Convert suspicious patterns into alert logic.

🧠 Examples
Detection 1: Excessive failed logons from a single source

index=main sourcetype=WinEventLog:Security EventCode=4625
| stats count by Source_Network_Address
| where count > 10


Purpose:
This query tests a simple detection rule: identify IP addresses that generate more than 10 failed logon attempts. This threshold is commonly used to detect brute-force login attempts or repeated credential misuse, especially if originating from a single source.

Result:
No results found.

Interpretation:
The absence of any IP addresses with more than 10 failed logon attempts confirms that there is no evidence of brute-force activity in the observed time range. The system continues to exhibit low-risk behavior, consistent with earlier Phase 2 findings.  


Detection 2: Privileged group membership outside business hours

index=main sourcetype=WinEventLog:Security EventCode=4728
| eval hour=strftime(_time,"%H")
| where hour<7 OR hour>19

Purpose:
This rule identifies attempts to add users to privileged groups (such as "Administrators") during off-hours. These changes, when performed outside of normal business hours, can indicate unauthorized privilege escalation or attacker lateral movement.

Result:
No results found.

Interpretation:
No group membership changes have occurred — during business hours or otherwise. This confirms that no privilege escalation behavior has been attempted on the system. The rule remains an effective check for future suspicious activity.

**Phase 3 Summary: Detection Logic Prototypes**

In Phase 3, we transitioned from investigation to action by prototyping simple detection logic that could be used in a real SOC environment to generate alerts. Detection A tested for excessive failed logon attempts from a single IP address — a common indicator of brute-force activity. The rule returned no matches, confirming that no source generated a suspicious volume of failed logons. Detection B searched for privilege group membership changes outside of business hours — a potential sign of unauthorized escalation. This rule also returned no matches, consistent with earlier findings that no group changes had taken place. Together, these rules demonstrate how targeted, rule-based logic can be layered on top of baseline and investigative findings to create real-time alerting mechanisms. Although no threats were found in this environment, the structure of these detections mirrors common industry practice and serves as a foundation for more complex behavioral or anomaly-based logic in future work.


**README.md Summary**

 Splunk Windows Security Log Analysis

Project Overview

This project investigates native Windows Security log data using Splunk to simulate real-world threat detection practices in a SOC (Security Operations Center) environment. The objective is to identify and understand system activity patterns, detect suspicious behaviors, and apply basic detection logic using SPL (Search Processing Language).

The analysis is structured into three progressive phases:

---

**Phase 1 – Baseline System Activity**

This phase establishes what "normal" looks like on the system by analyzing who logs in, when, from where, and using what type of logon. It provides a behavioral baseline to support later anomaly detection.

---

**Phase 2 – Suspicious Behavior Investigation**

Using targeted SPL filters, this phase investigates failed logons, account creation, privilege escalation, and off-hours logon activity. These checks are designed to uncover indicators of compromise (IOCs) or policy violations.

---

**Phase 3 – Detection Logic Prototypes**

This final phase converts suspicious patterns into detection rules. It includes simple logic to detect brute-force attempts and privilege changes outside business hours — both of which returned no alerts, consistent with the clean baseline.

---

**Outcome**

No signs of malicious activity were detected during this analysis. However, the methods used reflect standard practices in modern SOC environments and demonstrate how to move from raw data to operational detection logic using Splunk.

